# Limpeza de dados e estruturação

In [5]:
import numpy as np
import pandas as pd


In [6]:
from configyaml import load_config
config = load_config()

In [7]:
raw_data_path = f"../{config['directories']['raw_data']}/part-00000-87a9b3ed-0740-4af5-8394-4b8425895386.c000 (1).csv"
print(raw_data_path)

../data/raw/part-00000-87a9b3ed-0740-4af5-8394-4b8425895386.c000 (1).csv


In [8]:
#lista para colunas com IDENTIFICAÇÃO DO PACIENTE

dataPaciente = pd.read_csv(raw_data_path, sep=";", encoding="UTF-8")
print("Colunas disponíveis:", list(dataPaciente))


C:\Users\leoba\AppData\Local\Temp\ipykernel_27952\3869584644.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  dataPaciente = pd.read_csv(raw_data_path, sep=";", encoding="UTF-8")


Colunas disponíveis: ['sintomas', 'profissionalSaude', 'racaCor', 'outrosSintomas', 'outrasCondicoes', 'profissionalSeguranca', 'cbo', 'condicoes', 'sexo', 'estado', 'estadoIBGE', 'municipio', 'municipioIBGE', 'origem', 'estadoNotificacao', 'municipioNotificacao', 'municipioNotificacaoIBGE', 'evolucaoCaso', 'classificacaoFinal', 'codigoEstrategiaCovid', 'codigoBuscaAtivaAssintomatico', 'outroBuscaAtivaAssintomatico', 'codigoTriagemPopulacaoEspecifica', 'outroTriagemPopulacaoEspecifica', 'codigoLocalRealizacaoTestagem', 'outroLocalRealizacaoTestagem', 'codigoRecebeuVacina', 'codigoLaboratorioPrimeiraDose', 'codigoLaboratorioSegundaDose', 'lotePrimeiraDose', 'loteSegundaDose', 'codigoContemComunidadeTradicional', 'source_id', 'excluido', 'validado', 'codigoDosesVacina', 'estadoNotificacaoIBGE', 'totalTestesRealizados', 'dataNotificacao', 'dataInicioSintomas', 'dataEncerramento', 'dataPrimeiraDose', 'dataSegundaDose', 'codigoEstadoTeste1', 'codigoTipoTeste1', 'codigoFabricanteTeste1', 'co

## Dataframes para mapear os códigos da base principal 

In [132]:
dim_codigoLocalRealizacaoTestagem = pd.DataFrame({
    "codigoLocalRealizacaoTestagem": ["1", "2", "3", "4", "5"],
    "Descricao": [
        "Serviço de saúde (UBS, hospital, UPA etc.)",
        "Local de trabalho",
        "Aeroporto",
        "Farmácia ou drogaria",
        "Escola"
    ]
})

In [131]:
dim_codigoTriagemPopulacaoEspecifica = pd.DataFrame({
    "codigoTriagemPopulacaoEspecifica": ["1", "2", "3", "4", "5"],
    "Descricao": [
        "Trabalhadores de serviços essenciais ou estratégicos",
        "Profissionais de saúde",
        "Gestantes e puérperas",
        "Povos e comunidades tradicionais",
        "Outros"
    ]
})


In [130]:
dim_codigoEstrategiaCovid = pd.DataFrame({
    "codigoEstrategiaCovid": ["1", "2", "3"],
    "Descricao": [
        "Diagnóstico assistencial (sintomático)",
        "Busca ativa de assintomático",
        "Triagem de população específica"
    ]
})

In [129]:
dim_codigoRecebeuVacina = pd.DataFrame({
    "codigoRecebeuVacina": ["1", "2", "3"],
    "Descricao": ["Sim", "Não", "Ignorado"]
})


In [128]:
dim_codigoDosesVacina = pd.DataFrame({
    "codigoDosesVacina": ["1", "2"],
    "Descricao": ["1ª Dose", "2ª Dose"]
})


In [127]:
dim_codigoTipoTeste = pd.DataFrame({
    "codigoTipoTeste": ["1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "Descricao": [
        "RT-PCR",
        "RT LAMP",
        "Teste rápido de antígeno",
        "Teste rápido de anticorpo IgM",
        "Teste rápido de anticorpo IgG",
        "Teste sorológico IgA",
        "Teste sorológico IgM",
        "Teste sorológico IgG",
        "Teste sorológico – anticorpos totais"
    ]
})


In [126]:
dim_codigoEstadoTeste = pd.DataFrame({
    "codigoEstadoTeste": ["1", "2", "3", "4"],
    "Descricao": ["Solicitado", "Concluído", "Coletado", "Não Solicitado"]
})


In [125]:
dim_codigoResultadoTeste = pd.DataFrame({
    "codigoResultadoTeste": ["1", "2", "3"],
    "Descricao": [
        "Reagente/Detectável",
        "Não Reagente/Não Detectável",
        "Inválido/Inconclusivo ou Indeterminado"
    ]
})


In [124]:
dim_codigoBuscaAtivaAssintomatico = pd.DataFrame({
    "codigoBuscaAtivaAssintomatico": ["1", "2", "3", "4"],
    "Descricao": [
        "Monitoramento de contatos",
        "Investigação de surtos",
        "Monitoramento de viajantes com risco de VOC (quarentena)",
        "Outro"
    ]
})

In [120]:
dim_colunas = [
    "codigoRecebeuVacina",
    "codigoDosesVacina",
    "codigoTipoTeste1",
    "codigoEstadoTeste1",
    "codigoResultadoTeste1",
    "codigoBuscaAtivaAssintomatico"
]

dim_colunas_presentes = [col for col in dim_colunas if col in dataPaciente.columns]

for col in dim_colunas_presentes:
    dataPaciente[col] = dataPaciente[col].fillna(-1).astype(str)

print(dataPaciente[dim_colunas_presentes].dtypes)


codigoRecebeuVacina              object
codigoDosesVacina                object
codigoTipoTeste1                 object
codigoEstadoTeste1               object
codigoResultadoTeste1            object
codigoBuscaAtivaAssintomatico    object
dtype: object


In [133]:
df_final = pd.merge(
    dataPaciente,
    dim_codigoRecebeuVacina,
    on="codigoRecebeuVacina",
    how="left"
)

df_final = pd.merge(
    df_final,
    dim_codigoDosesVacina,
    on="codigoDosesVacina",
    how="left"
)

df_final = pd.merge(
    df_final,
    dim_codigoTipoTeste.rename(columns={"codigoTipoTeste": "codigoTipoTeste1"}),
    on="codigoTipoTeste1",
    how="left"
)

df_final = pd.merge(
    df_final,
    dim_codigoEstadoTeste.rename(columns={"codigoEstadoTeste": "codigoEstadoTeste1"}),
    on="codigoEstadoTeste1",
    how="left"
)

df_final = pd.merge(
    df_final,
    dim_codigoResultadoTeste.rename(columns={"codigoResultadoTeste": "codigoResultadoTeste1"}),
    on="codigoResultadoTeste1",
    how="left"
)

df_final = pd.merge(
    df_final,
    dim_codigoBuscaAtivaAssintomatico,
    on="codigoBuscaAtivaAssintomatico",
    how="left"
)

print(df_final)

C:\Users\leoba\AppData\Local\Temp\ipykernel_30784\3123684326.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Descricao_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_final = pd.merge(
C:\Users\leoba\AppData\Local\Temp\ipykernel_30784\3123684326.py:36: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Descricao_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_final = pd.merge(


                                           sintomas profissionalSaude  \
0                    Dispneia, Tosse, Dor de Cabeça               Não   
1                    Dispneia, Febre, Tosse, Coriza               Não   
2       Tosse, Dor de Cabeça, Distúrbios Gustativos               Não   
3                                     Dor de Cabeça               Não   
4                                     Tosse, Coriza               Não   
...                                             ...               ...   
466257                        Dor de Cabeça, Outros               Não   
466258                               Outros, Coriza               Não   
466259       Dor de Cabeça, Tosse, Dispneia, Coriza               Não   
466260       Coriza, Dor de Cabeça, Dor de Garganta               Não   
466261       Tosse, Dor de Garganta, Coriza, Outros               Não   

         racaCor                                     outrosSintomas  \
0          Preta                                    

In [134]:
processed_path = f"../{config['directories']['processed_data']}/df_processed.csv"
print(processed_path)
df_final.to_csv(processed_path, index=False, encoding="utf-8")

../data/processed/df_processed.csv
